<a href="https://colab.research.google.com/github/MsMore/Vaani/blob/main/finetunevgg3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os.path
import tensorflow as tf
from keras.callbacks import TensorBoard
from keras import Model, optimizers
from keras.applications import vgg16
from keras.layers import GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_width = 128
img_height = 128

epochs = 5
epochs_ft = 10
batch_size = 32

train_path = '/content/drive/MyDrive/Img/Train'
validation_path = '/content/drive/MyDrive/Img/Validation'

nb_classes = 29

In [ ]:
training_samples = sum([len(files) for r, d, files in os.walk(train_path)])
testing_samples = sum([len(files) for r, d, files in os.walk(validation_path)])

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.2,
    rescale=1.0 / 255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
)

test_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 10150 images belonging to 29 classes.
Found 2909 images belonging to 29 classes.


In [ ]:
def add_top(base):
    x = base.output

    # Global averaging pool layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Regular densely connected layer
    x = Dense(512, activation='relu')(x)

    # Output layer
    predictions = Dense(nb_classes, activation='softmax')(x)

    return Model(inputs=base.input, outputs=predictions)

In [ ]:
def setup_model(model, base):
    # Freeze the un-trainable layers of the model base
    for layer in base.layers:
        layer.trainable = False

    model.compile(
        loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy']
    )

In [ ]:
VGG_base = vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(img_width, img_height, 3),
    classes = 29
)

In [ ]:
VGG_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
model = add_top(VGG_base)
setup_model(model, VGG_base)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
!pip install tensorboardcolab

  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-cp37-none-any.whl size=3860 sha256=f3aa67e988e869ac779534d963a7c3183715199e4b92dd42c25363f34a0d5ccf
  Stored in directory: /root/.cache/pip/wheels/c4/aa/a0/3aaf4f1a66adbdab9b7bdd4c96db8ada89eb7cd87200cfdd32
Successfully built tensorboardcolab


In [ ]:
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = ModelCheckpoint('vgg16FT.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

In [ ]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://2b832d6524c3.ngrok.io


In [ ]:
!kill 430

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/VGG/pptlogs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 430), started 2:28:54 ago. (Use '!kill 430' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=testing_samples // batch_size,
    verbose=1,
    callbacks = [lr_reduce,checkpoint]
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
317/317 [==============================] - 7056s 22s/step - loss: 2.7072 - accuracy: 0.2856 - val_loss: 0.7749 - val_accuracy: 0.9240

Epoch 00001: val_accuracy improved from -inf to 0.92396, saving model to vgg16FT.h15
INFO:tensorflow:Assets written to: vgg16FT.h15/assets
Epoch 2/5
317/317 [==============================] - 62s 196ms/step - loss: 1.1729 - accuracy: 0.6934 - val_loss: 0.3593 - val_accuracy: 0.9389

Epoch 00002: val_accuracy improved from 0.92396 to 0.93889, saving model to vgg16FT.h15
INFO:tensorflow:Assets written to: vgg16FT.h15/assets
Epoch 3/5
317/317 [==============================] - 62s 197ms/step - loss: 0.7573 - accuracy: 0.7941 - val_loss: 0.2091 - val_accuracy: 0.9698

Epoch 00003: val_accuracy improved from 0.93889 to 0.96979, saving model to vgg16FT.h15
INFO:tensorflow:Assets written to: vgg16FT.h15/assets
Epoch 4/5
317/317 [==============================] - 62s 194ms/step - loss: 0.5634 - accuracy: 0.8411 - val_loss: 0.1969 - val_accuracy: 0.938

In [ ]:
def setup_model(model, trainable):
    # Freeze the un-trainable layers of the model base
    for layer in model.layers[:(len(model.layers) - trainable)]:  
        layer.trainable = False 

    for layer in model.layers[(len(model.layers) - trainable):]:
        layer.trainable = True

    model.compile(
        loss='categorical_crossentropy',
        # Slower training rate for fine-tuning
        optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
        metrics=['accuracy']
    )

In [ ]:
setup_model(model, 19)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples // batch_size,
    epochs=epochs_ft,
    validation_data=test_generator,
    validation_steps=testing_samples // batch_size,
    verbose=1,
    callbacks = [lr_reduce, checkpoint]
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
317/317 [==============================] - 65s 201ms/step - loss: 0.2932 - accuracy: 0.9068 - val_loss: 0.0195 - val_accuracy: 0.9920

Epoch 00001: val_accuracy improved from 0.98125 to 0.99201, saving model to vgg16FT.h15
INFO:tensorflow:Assets written to: vgg16FT.h15/assets
Epoch 2/10
317/317 [==============================] - 64s 201ms/step - loss: 0.1018 - accuracy: 0.9701 - val_loss: 0.0039 - val_accuracy: 1.0000

Epoch 00002: val_accuracy improved from 0.99201 to 1.00000, saving model to vgg16FT.h15
INFO:tensorflow:Assets written to: vgg16FT.h15/assets
Epoch 3/10
317/317 [==============================] - 64s 203ms/step - loss: 0.0783 - accuracy: 0.9779 - val_loss: 0.0017 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/10
317/317 [==============================] - 64s 200ms/step - loss: 0.0566 - accuracy: 0.9837 - val_loss: 0.0148 - val_accuracy: 0.9927

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/10
317/317 [==